In [ ]:
# some_file.py
#import sys
# insert at 1, 0 is the script path (or '' in REPL)
#sys.path.insert(1, '/Users/leon/Files/dscribe')
import dscribe
import dscribe.descriptors
#sys.path.insert(1, '/Users/leon/Files/dscribe/decriptors')
from dscribe.descriptors import SOAP
#from dscribe.descriptors import ACSF

In [ ]:
import numpy as np
import os
from tqdm import tqdm
from PIL import Image
from matplotlib.pyplot import imshow
import math
import sklearn
#import shap

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from collections import namedtuple
from ase.io import read
from ase.build import molecule
from ase import Atoms, Atom
from ase.visualize import view
from ase.geometry.analysis import Analysis
import plotly.graph_objects as go
from ase.data import vdw_radii
from ase.data.colors import cpk_colors, jmol_colors


In [ ]:
from soap_generation.alignment import align_elements
from soap_generation.augment import augment_elements

In [ ]:
import csv

barriers = dict()

with open('../data/vaskas_features_properties_smiles_filenames.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        #images.append(row[0])
        #elos.append(row[1])
        barriers[row[93]] = float(row[91])

In [ ]:
labels = []
elems = []
for f in tqdm(os.listdir("../data/coordinates_molSimplify/")):
    if f.endswith(".xyz"):
        if (f == "ir_tbp_1_dft-pet3_1_dft-py_1_dft-hicn_1_fluoride_1_smi1_1_s_1.xyz"):
            print("1:" + str(len(labels)))

        if (f == "ir_tbp_1_dft-pet3_1_dft-py_1_dft-hicn_1_dft-cn_1_smi1_1_s_1.xyz"):
            print("2:" + str(len(labels)))
        elems.append(read("../data/coordinates_molSimplify/" + f))
        labels.append(barriers[f[:-4]])

labels = np.array(labels)
number_samples = len(labels)

In [ ]:
elems = align_elements(elems)

In [ ]:
augment_steps = 30
elems, labels = augment_elements(elems, labels, steps=augment_steps)

In [ ]:
species = ["H","C","N","O","F","P","S","Cl","As","Br","I","Ir"]

rcut = 12.0
nmax = 8
lmax = 4

# Setting up the SOAP descriptor
soap = dscribe.descriptors.SOAP(
    species=species,
    periodic=False,
    rcut=rcut,
    nmax=nmax,
    lmax=lmax,
    rbf="gto"
)

In [ ]:
from decoder.soap import SOAPDecoder

decoder = SOAPDecoder(rcut=rcut, nmax=nmax, lmax=lmax)

In [ ]:
# Create soap coefficients
atom_index = [[0]] * len(elems)
features_soap = soap.create_coeffs(elems, positions=atom_index)

In [ ]:
# Scale coefficents
soapScaler = StandardScaler()
soapScaler.fit(features_soap)
features_soap = soapScaler.transform(features_soap)

In [ ]:
# Scale labels
labels = np.array(labels)
barrierScaler = StandardScaler()
barrierScaler.fit(labels.reshape(-1, 1))
labels = barrierScaler.transform(labels.reshape(-1, 1))
labels = np.array(labels)

In [ ]:
features_soap = features_soap.reshape(
    number_samples, augment_steps, -1)

labels = labels.reshape(
    number_samples, augment_steps, -1)

In [ ]:
def plot_density(coefficents, element=None, bond_width=0, resolution=10, opacity=0.1, atoms=None, is_diff=False, scale_limit=0):
    fig = go.Figure()
    
    X, Y, Z = np.mgrid[-10:10:(resolution * 1j), -10:10:(resolution * 1j), -10:10:(resolution * 1j)]
    values = np.zeros((resolution,resolution,resolution))
    
    indices = range(len(species))
    if atoms is not None:
        indices = [species.index(atoms)]
    
    for index in indices:
        values += decoder.density(coefficents, index, X, Y, Z)

    if scale_limit == 0:
        scale_limit = np.amax(np.abs(values))
    print(scale_limit)
    if is_diff:
        fig.add_trace(go.Volume(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=values.flatten(),
            isomin=-scale_limit,
            isomax=scale_limit,
            opacity=opacity, # needs to be small to see through all surfaces
            opacityscale=[(0, 1), (0.5, 0), (1, 1)],
            surface_count=30, # needs to be a large number for good volume rendering
            colorscale="RdBu"
            #hoverinfo='skip',
        ))
    else:
        fig.add_trace(go.Volume(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=values.flatten(),
            isomin=0,
            isomax=scale_limit,
            opacity=opacity, # needs to be small to see through all surfaces
            opacityscale=[(0, 0), (0.1, 0), (1, 1)],
            surface_count=30, # needs to be a large number for good volume rendering
            #hoverinfo='skip',
        ))

    
    if element is not None:
        for position, atomic_number in zip(element.get_positions(), element.get_atomic_numbers()):
            x1 = position[0]
            y1 = position[1]
            z1 = position[2]
            size = np.nan_to_num(vdw_radii[atomic_number]) * 0.3

            if size == 0:
                size = 0.5

            phi = np.linspace(0, 2*np.pi, 10)
            theta = np.linspace(-np.pi/2, np.pi/2, 10)
            phi, theta=np.meshgrid(phi, theta)

            x = np.cos(theta) * np.sin(phi) * size + x1
            y = np.cos(theta) * np.cos(phi) * size + y1
            z = np.sin(theta) * size + z1    
            color_string = "rgb(" + str(jmol_colors[atomic_number][0] * 230) + "," + str(jmol_colors[atomic_number][1] * 230) + "," + str(jmol_colors[atomic_number][2] * 230) + ")"

            fig.add_trace(go.Mesh3d({'x':x.flatten(), 
                                     'y':y.flatten(),
                                     'z':z.flatten(),
                                     'alphahull': 0, 
                                     'color': color_string,
                                    }))

        if bond_width > 0:
            bonds = Analysis(element).all_bonds[0]
            positions = element.get_positions()

            for index, bond in zip(range(len(positions)), bonds):
                x = []
                y = []
                z = []
                for atom in bond:
                    x.append(positions[index][0])
                    x.append(positions[atom][0])
                    x.append(positions[index][0])

                    y.append(positions[index][1])
                    y.append(positions[atom][1])
                    y.append(positions[index][1])

                    z.append(positions[index][2])
                    z.append(positions[atom][2])
                    z.append(positions[index][2])

                fig.add_trace(go.Scatter3d(
                    x = x,
                    y = y,
                    z = z,
                    mode = "lines",
                    line =
                        go.Line(
                            color = "rgb(10, 10, 10)",
                            width = bond_width
                        )
                ))

    fig.update_layout(
        height=1000,
        showlegend=False, 
        scene=dict(
            xaxis=dict(showticklabels=False, visible=False),
            yaxis=dict(showticklabels=False, visible=False),
            zaxis=dict(showticklabels=False, visible=False),
        ))
    fig.show()

In [ ]:
sample = 807
rotation = 0

coefficents = soapScaler.inverse_transform(features_soap[sample][0].flatten())

element = elems[sample * 20 + 10]
print(element)
plot_density(coefficents, element, bond_width=4, resolution=40, opacity=0.4)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(
        features_soap, labels, test_size=0.2, random_state=32)

(testX, valX, testY, valY) = train_test_split(
        testX, testY, test_size=0.5, random_state=32)

In [ ]:
dir = "/pfs/work7/workspace/scratch/utpqw-data-0/bachelor-thesis/bachelor-thesis/code/"
sample = "9:3:0.2"
trainX = np.load(dir + "features_train_" + sample + ".npy")
trainY = np.load(dir + "labels_train_" + sample + ".npy")

valX = np.load(dir + "features_val_" + sample + ".npy")
valY = np.load(dir + "labels_val_" + sample + ".npy")

testX = np.load(dir + "features_test_" + sample + ".npy")
testY = np.load(dir + "labels_test_" + sample + ".npy")

In [ ]:
trainX = trainX.reshape(-1, 12, int(trainX.shape[2] / 12), 1)
testX = testX.reshape(-1, 12, int(testX.shape[2] / 12), 1)
valX = valX.reshape(-1, 12, int(valX.shape[2] / 12), 1)
trainY = trainY.flatten()
testY = testY.flatten()
valY = valY.flatten()

In [ ]:
def reg_stats(y_true,y_pred,scaler=None):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  if scaler:
    y_true_unscaled = scaler.inverse_transform(y_true)
    y_pred_unscaled = scaler.inverse_transform(y_pred)
  r2 = sklearn.metrics.r2_score(y_true,y_pred)
  mae = sklearn.metrics.mean_absolute_error(y_true_unscaled, y_pred_unscaled)
  return r2,mae

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 0.5])
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
def get_model(hp):
    input_shape = trainX[0].shape
    inputs = tf.keras.Input(shape=input_shape)

    x = inputs
    x = tf.keras.layers.Flatten()(x)

    for i in range(hp.Int('hidden_layers', 1, 6, default=3)):
        size = hp.Int('hidden_size_' + str(i), 10, 700, step=40)
        reg = hp.Float('hidden_reg_' + str(i), 0,
                       0.06, step=0.01, default=0.02)
        dropout = hp.Float('hidden_dropout_' + str(i),
                           0, 0.5, step=0.1, default=0.2)

        x = tf.keras.layers.Dense(size, activation="relu",
                                  kernel_regularizer=regularizers.l2(reg))(x)
        x = tf.keras.layers.Dropout(dropout)(x)

        norm = hp.Choice('hidden_batch_norm_' + str(i), values=[True, False])

        if norm:
            x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dense(1, kernel_regularizer='l2')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-6, 1e-4, sampling='log')),
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.MeanSquaredError()])

    return model

In [ ]:
from kerastuner.tuners import Hyperband
import kerastuner as kt

tuner = kt.Hyperband(
    get_model,
    objective='val_mean_squared_error',
    max_epochs=1200,
    project_name="Hyperband_FINAL_SNAP_9:3:0.4",
    directory="/pfs/work7/workspace/scratch/utpqw-data-0/bachelor-thesis/bachelor-thesis/code"
)

In [ ]:
split = 0.2

trainX = np.concatenate((trainX, valX))
trainY = np.concatenate((trainY, valY))

(trainX, valX, trainY, valY) = train_test_split(
        trainX, trainY, test_size=split / 0.8, random_state=32)

In [ ]:
tuner.results_summary()

In [ ]:
model_full = get_model(tuner.get_best_hyperparameters(4)[3])

In [ ]:
from keras.utils.vis_utils import plot_model, model_to_dot
import visualkeras

model_full.summary()
visualkeras.layered_view(model_full).show()
#plot_model(model_full, show_shapes=True, show_layer_names=True)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=tuner.get_best_hyperparameters(1)[0]["learning_rate"])

model_full.compile(loss="mean_squared_error", optimizer=opt)

In [ ]:
H = model_full.fit(x=trainX, y=trainY, validation_data=(valX, valY), epochs=2000, batch_size=400, verbose=2, callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=80)])
plot_loss(H)

In [ ]:
r2_train, mae_train = reg_stats(trainY, model_full.predict(trainX), barrierScaler)
r2_val, mae_val = reg_stats(valY, model_full.predict(valX), barrierScaler)
r2_test, mae_test = reg_stats(testY, model_full.predict(testX), barrierScaler)
print("R^2 Train: " + str(r2_train))
print("MAE Train: " + str(mae_train))
print("")
print("R^2 Validation: " + str(r2_val))
print("MAE Validation: " + str(mae_val))
print("")
print("R^2 Test: " + str(r2_test))
print("MAE Test: " + str(mae_test))

train_y_pred = barrierScaler.inverse_transform(model_full.predict(trainX))
train_y_real = barrierScaler.inverse_transform(trainY)

val_y_pred = barrierScaler.inverse_transform(model_full.predict(valX))
val_y_real = barrierScaler.inverse_transform(valY)

test_y_pred = barrierScaler.inverse_transform(model_full.predict(testX))
test_y_real = barrierScaler.inverse_transform(testY)


fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(train_y_real, train_y_pred, marker="o", c="C1", label="Training")
ax.scatter(val_y_real, val_y_pred, marker="o", c="C3", label="Validation")
ax.scatter(test_y_real, test_y_pred, marker="o", c="C2", label="Testing")
ax.set_aspect('equal')
ax.set_xlabel("Calculated barrier [kcal/mol]")
ax.set_ylabel("Predicted barrier [kcal/mol]")
ax.legend(loc="upper left")

plt.savefig("../figs/scatter_" + str(nmax) + "-" + str(lmax) + "-" + str(split) + ".png")
plt.show()


file = open("../figs/out_hyperparam.csv", "a")
file.write(str(30))
file.write(",")
file.write(str(split))
file.write(",")
file.write(str(nmax))
file.write(",")
file.write(str(lmax))
file.write(",")
file.write(str(rcut))
file.write(",")
file.write(str(r2_test))
file.write(",")
file.write(str(mae_test))
file.write("\n")
file.close()


In [ ]:
#model_full.save("conv_20:3:3.h5")

In [ ]:
model_full = keras.models.load_model("model_8-4.h5")

In [ ]:
element_rotations, labels_rotations = augment_elements([elems[665]], labels, steps=360)
atom_index = [[0]] * len(element_rotations)
features_rotations = soap.create_coeffs(element_rotations, positions=atom_index)

features_rotations = soapScaler.transform(features_rotations)

plt.imshow(features_rotations[0].reshape(12,200))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.ylabel('$Z$')
plt.xlabel('$c^Z_{nlm}$')
plt.title('Rotation 0°')
plt.show()

plt.imshow(features_rotations[5].reshape(12,200))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.ylabel('$Z$')
plt.xlabel('$c^Z_{nlm}$')
plt.title('Rotation 90°')
plt.show()

plt.imshow(features_rotations[10].reshape(12,200))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.xlabel('$c^Z_{nlm}$')
plt.ylabel('$Z$')
plt.title('Rotation 180°')
plt.show()

plt.imshow(features_rotations[15].reshape(12,200))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.ylabel('$Z$')
plt.xlabel('$c^Z_{nlm}$')
plt.title('Rotation 270°')
plt.show()

In [ ]:

model_full = keras.models.load_model("/Users/leon/out/model____augment_steps=30_l=4_n=8_split=0.3_rcut=12.h5")

#scaler = copy.deepcopy(soapScaler)

nmax = 8
lmax = 4

# Setting up the SOAP descriptor
soap = dscribe.descriptors.SOAP(
    species=species,
    periodic=False,
    rcut=rcut,
    nmax=nmax,
    lmax=lmax,
    rbf="gto"
)

samples = [0, 74, 8, 898]
colors = ["C1", "C2", "C3", "C4"]

for sample, color in zip(samples, colors):
    rotated, _ = augment_elements([elems[sample * augment_steps]], [0] * 3000, steps=360)
    print(len(rotated))
    
    atom_index = [[0]] * len(rotated)
    
    species = ["H","C","N","O","F","P","S","Cl","As","Br","I","Ir"]

    rcut = 12.0

    feature_rotations = soap.create_coeffs(rotated, positions=atom_index)
    feature_rotations = soapScaler.transform(feature_rotations)
    
    feature_rotations = feature_rotations.reshape(-1, 12, int(feature_rotations.shape[1] / 12), 1)
    
    print(feature_rotations.shape)
    predictions = model_full.predict(feature_rotations)

    predictions = barrierScaler.inverse_transform(predictions)
   
    plt.step(np.arange(0,360), predictions, color=color, label=elems[sample].symbols)
    
    label = [labels[sample][0]] * 360
    plt.step(np.arange(0,360), barrierScaler.inverse_transform(label), ":", color=color)

    #plt.step(np.arange(0,360), [barrierScaler.inverse_transform(labels[sample][0])] * 360, label='real')

steps = np.linspace(0, 360, 31)

for step in steps:
    plt.vlines(x=step, ymin=0, ymax=100, colors="k", linestyle=":", linewidth=0.6)
    
plt.grid(axis='x', color='0.95')
plt.xlabel('Rotation [degrees]')
plt.ylabel('Prediction [kcal/mol]')
plt.ylim([8,19.5])

steps = np.linspace(0, 360, 11)
plt.xticks(steps)
#plt.legend()
#plt.title('Prediction over rotation')
plt.show()

In [ ]:
view(rotated[60], viewer='x3d')

In [ ]:
import shap
shap.initjs()

In [ ]:
for layer in model_full.layers:
    layer.trainable = True

In [ ]:
shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough

# select a set of background examples to take an expectation over
background = trainX[np.random.choice(trainX.shape[0], 100, replace=False)]
print(background.shape)
# explain predictions of the model on four images
e = shap.DeepExplainer(model_full, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)

features_soap = features_soap.reshape(-1, 12, int(features_soap.shape[2] / 12), 1)
selection = [0,20,40,60]
shap_values = e.shap_values(features_soap[selection])
#print(shap_values.shape)
# plot the feature attributions
shap.image_plot(shap_values, -features_soap[selection])

In [ ]:
print(np.sum(shap_values[0][0]).reshape(-1,1))
barrierScaler.inverse_transform(np.sum(shap_values[0][0]).reshape(-1,1))

In [ ]:
print(np.array(shap_values).shape)

In [ ]:
plt.matshow(features_soap[selection[0]].reshape(12,48))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.colorbar()
plt.title("Features")
plt.show()

In [ ]:
plt.matshow(shap_values[0][0].reshape(12,48))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.colorbar()
plt.title("SHAP Values")
plt.show()

In [ ]:
shap_scaled = barrierScaler.inverse_transform(shap_values[0][0].flatten())
element = elems[selection[0]]
imshow(shap_scaled.reshape(12,48))

In [ ]:
def plot_density_diff(shap, coefficents, scaler, element=None, bond_width=0, resolution=10, opacity=0.1, atoms=None, scale_limit=0):  
    sc_scaled = scaler.inverse_transform(coefficents + shap)
    c_scaled = scaler.inverse_transform(coefficents)
    coeffs = sc_scaled - c_scaled
    
    plot_density(coeffs, element, bond_width, resolution, opacity, atoms, is_diff=True, scale_limit=scale_limit)

In [ ]:
plot_density(
    shap_scaled, 
    element, 
    bond_width=4, 
    opacity=0.8, 
    resolution=20, 
    atoms="As"
)

In [ ]:
print(element)

print(features_soap[selection[0]].shape)
plot_density_diff(
    shap_values[0][0].flatten(), 
    features_soap[selection[0]].flatten(), 
    soapScaler, 
    element, 
    bond_width=4, 
    opacity=0.3, 
    resolution=40, 
    atoms="As"
)

In [ ]:
element_contrib = np.sum(shap_values[0][0], axis = 1).flatten()
plt.bar(list(range(len(species))), element_contrib)
plt.ylabel("Influence on prediction")
plt.xlabel("Elements")
plt.xticks(list(range(len(species))), species)
plt.title("Influence of features for species")
plt.show()

# Gradients

In [ ]:
model_full = keras.models.load_model("model_8-4.h5")

In [ ]:
index = 979#357 #807#979
print(barrierScaler.inverse_transform(labels[807])[0])
element = elems[index * 30 + 15]

print(features_soap[index].shape)
print(element)

image = features_soap[index][0].reshape((12, int(features_soap[index].shape[1] / 12)))

plt.imshow(image)
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.show()

#explainer.save(grid, '.', 'act.png')

In [ ]:
zero_scaled = barrierScaler.transform(np.array([[0]]))[0]
print(zero_scaled)

with tf.GradientTape() as tape:
    inputs = tf.cast(image.reshape((1, 12, int(features_soap[index].shape[1] / 12), 1)), tf.float32)
    tape.watch(inputs)
    prediction = model_full(inputs)

    print("Prediction: " + str(prediction))

grid = tape.gradient(prediction, inputs).numpy()
print(grid.shape)

plt.imshow(grid.reshape(12,int(features_soap[index].shape[1] / 12)))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.show()


In [ ]:
def compute_gradient(model, sample, epsilon=1):
    gradient = np.zeros(sample.flatten().shape)
    
    # Scedule all gradients to be computed
    gradients = []
    for index in range(len(gradient)):
        direction = np.zeros(sample.flatten().shape)
        direction[index] = epsilon
        
        gradients += [(sample.flatten() + direction).reshape(sample.shape)]
    
    
    gradients = np.array(gradients)
    new_shape = [-1] + list(sample.shape)

    predictions = model.predict(gradients.reshape(new_shape))
    
    value = model.predict(np.array([sample]))
    
    gradients = (predictions - value) / epsilon
    
    return gradients.reshape(sample.shape)

In [ ]:
grid = compute_gradient(model_full, image.reshape(12,200), epsilon=3).reshape(1,12,200,1)

plt.imshow(grid.reshape(12,200))
y_axis = np.arange(0, 12, 1)
plt.yticks(y_axis, species)
plt.yticks(fontsize=7)
plt.show()

In [ ]:
print(element)
plot_density_diff(grid.flatten(), image.flatten(), soapScaler, element, bond_width=4, opacity=1, resolution=40, atoms="N")
#0.0013641491280516759

In [ ]:
resolution = 20
influence = []
for atom in range(len(species)):
    X, Y, Z = np.mgrid[-10:10:(resolution * 1j), -10:10:(resolution * 1j), -10:10:(resolution * 1j)]
    values = np.zeros((resolution,resolution,resolution))

    indices = range(len(species))

    sc_scaled = soapScaler.inverse_transform(image.flatten() + grid.flatten())
    c_scaled = soapScaler.inverse_transform(image.flatten())

    values += decoder.density(sc_scaled, atom, X, Y, Z) - decoder.density(c_scaled, atom, X, Y, Z)

    influence.append(np.sum(values))
    


plt.bar(list(range(len(species))), influence)
plt.ylabel("Gradient density")
plt.xlabel("Elements")
plt.xticks(list(range(len(species))), species)
plt.title("$\sum \rho_\nabla^Z$gradient density space")
plt.ylim([-0.5,0.1])
plt.show()

In [ ]:
zero_scaled = barrierScaler.transform(np.array([[0]]))[0]
print(zero_scaled)

influence_global = np.zeros(12)

print(len(testX))
for sample in testX:
    print(sample.shape)
    with tf.GradientTape() as tape:
        inputs = tf.cast(sample[0].reshape((1, 12, int(sample.shape[1] / 12), 1)), tf.float32)
        tape.watch(inputs)
        prediction = model_full(inputs)

        print("Prediction: " + str(prediction))

    grid = tape.gradient(prediction, inputs).numpy()

    resolution = 10
    influence = []
    for atom in range(len(species)):
        X, Y, Z = np.mgrid[-10:10:(resolution * 1j), -10:10:(resolution * 1j), -10:10:(resolution * 1j)]
        values = np.zeros((resolution,resolution,resolution))

        indices = range(len(species))

        sc_scaled = soapScaler.inverse_transform(image.flatten() + grid.flatten())
        c_scaled = soapScaler.inverse_transform(image.flatten())

        values += decoder.density(sc_scaled, atom, X, Y, Z) - decoder.density(c_scaled, atom, X, Y, Z)

        influence.append(np.sum(values))
    
    influence_global += np.array(influence)


plt.bar(list(range(len(species))), influence_global / len(testX))
plt.ylabel("Gradient density")
plt.xlabel("Elements")
plt.xticks(list(range(len(species))), species)
plt.title("Integration of gradient density space")
plt.show()

# Interpolation

In [ ]:
element1 = features_soap[123][0] #809
element2 = features_soap[147][15] #979

def interpolate(alpha):
    return (element1 * alpha) + (element2 * (1 - alpha))

In [ ]:
predictions = []
for i in range(15):
    for j in range(15):
        element1 = features_soap[369][i] #809
        element2 = features_soap[456][j] 
        
        interpolations = []
        for x in np.linspace(0,1,50):
            interpolations.append(interpolate(x))

        interpolations = np.array(interpolations).reshape(-1,12,200,1)
        
        predictions.append(barrierScaler.inverse_transform(model_full.predict(interpolations)))

In [ ]:
for prediction in predictions:
    plt.plot(np.linspace(0,1,50), prediction, "-")

plt.xlabel('Interpolation [$ x $]')
plt.ylabel('MAE [$kcal/mol$]')
plt.show()